In [1]:
!pip install textblob
!pip install texthero
import textblob
import string

## for data
import pandas as pd
import collections
import json
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud
## for text processing
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
## for sentiment
from textblob import TextBlob
## for ner
import spacy
from spacy.matcher import Matcher     
from spacy.util import filter_spans

# en_core_web_lg.load()
# spacy.load('en_core_web_lg')
!python -m spacy download en_core_web_lg 
# nlp = spacy.load('en_core_web_lg')
import en_core_web_lg
nlp = en_core_web_lg.load()
## for vectorizer
from sklearn import feature_extraction, manifold
## for word embedding
import gensim.downloader as gensim_api
## for topic modeling
import gensim

from sklearn.linear_model import LogisticRegression
import texthero as hero
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack
import pickle
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score 

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')

     |████████████████████████████████| 1.5 MB 7.7 MB/s 
     |████████████████████████████████| 235 kB 49.9 MB/s 
     |████████████████████████████████| 748 kB 48.2 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


     |████████████████████████████████| 827.9 MB 1.3 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=8718610957a2ea251f0f5782929ba4b8da8d1b0f487b007bbaab4a11f51abc3f
  Stored in directory: /tmp/pip-ephem-wheel-cache-1n2zjhad/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [53]:
o_df = pd.read_csv('/content/drive/MyDrive/clickbait_data.csv')
df=o_df.sample(frac=0.8,random_state=42) #random state is a seed value
test_df=o_df.drop(df.index)

# Get the lists of sentences and their labels.
headlines = df.headline.values
labels = df.clickbait.values

test_headlines = test_df.headline.values
test_labels = test_df.clickbait.values

In [54]:
vect = CountVectorizer(analyzer='word', ngram_range=(1, 3),max_features=300000)
vect.fit(o_df["headline"])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=300000, min_df=1,
                ngram_range=(1, 3), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [55]:
clf = LogisticRegression(solver="saga",class_weight={0:0.5,1:0.5},l1_ratio=0.5,penalty="elasticnet",random_state=42)

In [56]:
vectorized_x = vect.transform(headlines)

In [57]:
clf.fit(vectorized_x, labels)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



LogisticRegression(C=1.0, class_weight={0: 0.5, 1: 0.5}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=0.5,
                   max_iter=100, multi_class='auto', n_jobs=None,
                   penalty='elasticnet', random_state=42, solver='saga',
                   tol=0.0001, verbose=0, warm_start=False)

In [58]:
vectorized_x_test = vect.transform(test_headlines)
predictions = clf.predict(vectorized_x_test)

Run#4 Char tri-grams, max-features=300k

In [51]:
print("Size of testing data:", len(predictions))
true_labels = test_labels
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score 
acc=accuracy_score(predictions,true_labels)
f1=f1_score(predictions,true_labels)
precision=precision_score(predictions,true_labels)
recall=recall_score(predictions,true_labels)
print(" accuracy={} \n f1={} \n precision={} \n recall={}".format(acc,f1,precision,recall))

Size of testing data: 6400
 accuracy=0.96546875 
 f1=0.9643950378604801 
 precision=0.9614519755862512 
 recall=0.9673561732385262


In [52]:
all_columns = list(vect.get_feature_names())
coefficient_map = list(zip(all_columns, list(clf.coef_[0])))
coefficient_map.sort(key= lambda x:x[1],reverse=True)
coefficient_map

[('1', 1.0289885898753333),
 ('9 ', 0.8742370872398567),
 ('2', 0.869036189788167),
 ('thi', 0.864924680908733),
 ('"', 0.8435198489872551),
 ('lov', 0.8334122399486203),
 ('his', 0.7393097235616517),
 ('hat', 0.7172619581774183),
 ("'s ", 0.7122890287954581),
 ('you', 0.6839251764035631),
 ('ou ', 0.6405164053289868),
 ('rea', 0.6269262460265843),
 ('is ', 0.6266781887355943),
 ('jus', 0.6249182940535825),
 ('yo', 0.6208520755040141),
 ('sn', 0.6113998322903028),
 ("'s", 0.6062010606598134),
 ('tru', 0.6048973514079514),
 (' yo', 0.6029535884780423),
 ('7 ', 0.6028484812422791),
 ('ese', 0.5953097113901968),
 (' an', 0.5776699103813798),
 ('15', 0.570458834736041),
 ('s o', 0.5654968688978311),
 ('uck', 0.5636166449303235),
 ('ple', 0.5630380340343039),
 ('wil', 0.5578085533623219),
 ('ed ', 0.5574795147555557),
 ('eo', 0.548768781556787),
 ('ute', 0.5486983467142185),
 ('nde', 0.5400423204238626),
 ('ram', 0.5343232213814109),
 ('ans', 0.5322691950959081),
 ('bou', 0.5289157250786138

Run#3 Char bi-grams, max-features=100k

In [43]:
print("Size of testing data:", len(predictions))
true_labels = test_labels
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score 
acc=accuracy_score(predictions,true_labels)
f1=f1_score(predictions,true_labels)
precision=precision_score(predictions,true_labels)
recall=recall_score(predictions,true_labels)
print(" accuracy={} \n f1={} \n precision={} \n recall={}".format(acc,f1,precision,recall))

Size of testing data: 6400
 accuracy=0.93828125 
 f1=0.9361977063479244 
 precision=0.9309347895920335 
 recall=0.9415204678362573


In [44]:
all_columns = list(vect.get_feature_names())
coefficient_map = list(zip(all_columns, list(clf.coef_[0])))
coefficient_map.sort(key= lambda x:x[1],reverse=True)
coefficient_map

[('yo', 1.609948589629),
 ('1', 1.306970917354339),
 ('9 ', 1.265894278463467),
 ("'s", 1.2452123218507318),
 ('eo', 1.2125374037658265),
 ('sn', 1.1975313640353786),
 ('tc', 1.078087014915399),
 ('15', 1.0769694153636726),
 ('ab', 1.0680078566736197),
 ('2', 1.002674524204666),
 ('0s', 0.9977061149573807),
 ('rf', 0.9748205932733223),
 ('7 ', 0.9748168341321323),
 ('ly', 0.9477166273200637),
 ('"', 0.9352317498817952),
 ('yn', 0.8382683646431867),
 ('17', 0.7820542056622101),
 ("e'", 0.7601211518652876),
 ('4 ', 0.7504566491187558),
 ('u ', 0.748054212986454),
 ('um', 0.7358824813614382),
 ('fu', 0.7294602339205655),
 ('vs', 0.7273997396213026),
 ('6 ', 0.7252741803002433),
 ('hi', 0.7235369410160254),
 ('8 ', 0.7182473189927703),
 ('if', 0.718245433125479),
 ("t'", 0.6933682895571056),
 (' q', 0.6893679507419156),
 ('16', 0.6798977200779245),
 ('ha', 0.6711025389828564),
 ('ok', 0.645419344176657),
 ('18', 0.6340849190420514),
 ('br', 0.6289457621069046),
 (' g', 0.627299143353105),


Run#2 Word tri-grams, max-features=300k

In [59]:
print("Size of testing data:", len(predictions))
true_labels = test_labels
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score 
acc=accuracy_score(predictions,true_labels)
f1=f1_score(predictions,true_labels)
precision=precision_score(predictions,true_labels)
recall=recall_score(predictions,true_labels)
print(" accuracy={} \n f1={} \n precision={} \n recall={}".format(acc,f1,precision,recall))

Size of testing data: 6400
 accuracy=0.9703125 
 f1=0.9691458265670672 
 precision=0.95856087375522 
 recall=0.9799671592775041


In [60]:
all_columns = list(vect.get_feature_names())
coefficient_map = list(zip(all_columns, list(clf.coef_[0])))
coefficient_map.sort(key= lambda x:x[1],reverse=True)
coefficient_map

[('you', 3.58775422763066),
 ('these', 3.0724062455098653),
 ('this', 3.008900492021325),
 ('things', 2.7320845607921873),
 ('here', 2.6858030932212102),
 ('2015', 2.656399469528201),
 ('21', 2.6475714229732725),
 ('17', 2.601807009472537),
 ('your', 2.57415537516641),
 ('but not', 2.572239349930802),
 ('buzzfeed', 2.5482795120461343),
 ('everyone', 2.4830800529347083),
 ('confessions', 2.478885287195922),
 ('18', 2.4776047763110065),
 ('why', 2.3500035628483644),
 ('my', 2.3337903421756905),
 ('19', 2.294040955928442),
 ('got', 2.2876980105157507),
 ('what', 2.2592642123245352),
 ('there', 2.242432469775872),
 ('every', 2.219349828039351),
 ('23', 2.1723701079713016),
 ('halloween', 2.141901235759247),
 ('perfect', 2.0862837840369615),
 ('people', 2.084906683158539),
 ('actually', 2.077463432625627),
 ('we', 2.0680915907523927),
 ('15', 1.9983909224879102),
 ('jenner', 1.9866357803948822),
 ('22', 1.9669153653480018),
 ('14', 1.9654656507726878),
 ('someone', 1.9265797124123605),
 ('g

In [67]:
print("Words commonly used with clickbait headlines:")
coefficient_map[:5]


Words commonly used with clickbait headlines:


[('you', 3.58775422763066),
 ('these', 3.0724062455098653),
 ('this', 3.008900492021325),
 ('things', 2.7320845607921873),
 ('here', 2.6858030932212102)]

In [66]:
print("Words commonly used with non-clickbait headlines:")
coefficient_map[-10:-3]


Words commonly used with non-clickbait headlines:


[('2008', -1.5211558141686314),
 ('uk', -1.5453662433942446),
 ('000', -1.5620529483286123),
 ('australian', -1.5621161500962764),
 ('obama', -1.575520682142606),
 ('mets', -1.599151785871157),
 ('dies', -1.6057024200171233)]

Run#1 Word bi-grams, max-features=100k

In [10]:
print("Size of testing data:", len(predictions))
true_labels = test_labels
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score 
acc=accuracy_score(predictions,true_labels)
f1=f1_score(predictions,true_labels)
precision=precision_score(predictions,true_labels)
recall=recall_score(predictions,true_labels)
print(" accuracy={} \n f1={} \n precision={} \n recall={}".format(acc,f1,precision,recall))

Size of testing data: 6400
 accuracy=0.9696875 
 f1=0.9684860298895387 
 precision=0.9575971731448764 
 recall=0.979625369700953


In [26]:
all_columns = list(vect.get_feature_names())
coefficient_map = list(zip(all_columns, list(clf.coef_[0])))
coefficient_map.sort(key= lambda x:x[1],reverse=True)
coefficient_map

[('you', 3.599624196643321),
 ('these', 3.093274711331848),
 ('this', 3.0236370202381195),
 ('things', 2.7528442565306293),
 ('here', 2.7071044851049035),
 ('2015', 2.6983505769047618),
 ('21', 2.6691186477239346),
 ('but not', 2.658512412879477),
 ('17', 2.6305848695204586),
 ('buzzfeed', 2.6235206725085205),
 ('your', 2.600011025855501),
 ('confessions', 2.5449246873859446),
 ('18', 2.50005739114117),
 ('everyone', 2.4972699539417045),
 ('why', 2.390254402440896),
 ('my', 2.370483680283671),
 ('19', 2.315435017923845),
 ('got', 2.3098541088439957),
 ('what', 2.285473753477761),
 ('there', 2.26406267025022),
 ('every', 2.234023349778709),
 ('23', 2.1910266971283803),
 ('halloween', 2.1728507026527324),
 ('perfect', 2.125477493395231),
 ('people', 2.1004772417277513),
 ('actually', 2.0984361973219103),
 ('we', 2.074106932323747),
 ('jenner', 2.046105035261469),
 ('15', 2.0225114219363447),
 ('22', 1.9926310082352299),
 ('14', 1.9783876810599312),
 ('guys', 1.9692266411048571),
 ('try',

In [69]:
df = pd.read_csv("/content/drive/MyDrive/00FINAL_TRUEUpworthyWithUpworthinessColumn.csv")
df = df.drop_duplicates(subset='headline', keep="last")

In [70]:
df

,created_at,test_week,test_id,headline,image_id,excerpt,lede,slug,share_text,share_image,impressions,clicks,first_place,winner,click_rate,upworthiness
12,2014-11-20,2014-11-17,546e009a9ad54ec65b00004b,What They Learned From The Scientist Was Terri...,546c7f2dbadeb5788700000a,Things that matter. Pass 'em on.,<p>Some decent guys fighting for an important ...,what-they-learned-from-the-scientist-was-terri...,NaN,NaN,4594,51,False,False,0.011101,0.999804
23,2014-11-20,2014-11-17,546e009a9ad54ec65b00004b,He Sat Them Down And Told Them About An Immine...,546c7f2dbadeb5788700000a,Things that matter. Pass 'em on.,<p>Some decent guys fighting for an important ...,he-sat-them-down-and-told-them-about-an-immine...,NaN,NaN,4601,27,False,False,0.005868,0.999817
25,2014-11-20,2014-11-17,546e009a9ad54ec65b00004b,"They May Not Be The Most Handsome Dudes, But T...",546c7f2dbadeb5788700000a,Things that matter. Pass 'em on.,<p>Some decent guys fighting for an important ...,they-may-not-be-the-most-handsome-dudes-but-th...,NaN,NaN,4524,44,False,False,0.009726,0.999516
27,2014-11-20,2014-11-17,546cb9c954524053ac000013,Warning: Beware Of Her Selfies And Stereotypes...,546c7f42badeb54dd0000005,Things that matter. Pass 'em on.,<p>Africa is not your excuse to take selfies.</p>,warning-beware-of-her-selfies-and-stereotypes-...,The sad thing is that she thinks she's helping.,NaN,4600,154,False,False,0.033478,0.999795
28,2014-11-20,2014-11-17,546cb9c954524053ac000013,"Sigh... I Wanted To Like This Woman, But The S...",546c7f42badeb54dd0000005,Things that matter. Pass 'em on.,<p>Africa is not your excuse to take selfies.</p>,sigh-i-wanted-to-like-this-woman-but-the-selfi...,The sad thing is that she thinks she's helping.,NaN,4603,162,False,False,0.035194,0.999750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150797,2014-11-20,2014-11-17,546d414a84ad380752000026,"The More And More These 4 People Talk, The Mor...",546cae83badeb5294a000023,But it should be everywhere.,<p>I know our inclination is to skip over heav...,the-more-and-more-these-4-people-talk-the-more...,NaN,NaN,3174,32,False,False,0.010082,0.999734
150803,2014-11-20,2014-11-17,546d442d84ad3834f0000027,"The More And More These 4 People Talk, The Mor...",5332fb36a2fc90ead000217d,This video should be everywhere.,<p>I know our inclination is to skip over heav...,the-more-and-more-these-4-people-talk-the-more...,NaN,NaN,4083,81,False,False,0.019838,0.999638
150808,2014-11-20,2014-11-17,546d4e139ad54eec8d000019,Say Goodbye To The News You Once Knew. These 2...,546a84cc5b606a1fa0000022,Things that matter. Pass 'em on.,<p>This is a pretty great idea for a TV show! ...,say-goodbye-to-the-news-you-once-knew-these-2-...,What if a news show wants to do the opposite o...,NaN,4120,21,False,False,0.005097,0.999769
150812,2014-11-20,2014-11-17,546d564284ad3817bc000025,An Artist Animates The Ugly Truth We Should Al...,546d574a9ad54e2af0000020,Things that matter. Pass 'em on.,<p>Sad but true: It took the deaths of several...,an-artist-animates-the-ugly-truth-we-should-al...,NaN,NaN,10011,53,False,False,0.005294,0.999708
